Mounted at /content/drive


In [29]:
# Let's first load the datasets to understand their structure
import pandas as pd

# Loading the fake news dataset
fake_news_df = pd.read_csv("E:\Code\dataset\gossipcop_fake.csv")

# Loading the real news dataset
real_news_df = pd.read_csv("E:\Code\dataset\gossipcop_real.csv")

# Displaying the first few rows of each dataset to observe the data
fake_news_df.head(), real_news_df.head()


(                     id                                           news_url  \
 0  gossipcop-2493749932  www.dailymail.co.uk/tvshowbiz/article-5874213/...   
 1  gossipcop-4580247171  hollywoodlife.com/2018/05/05/paris-jackson-car...   
 2   gossipcop-941805037  variety.com/2017/biz/news/tax-march-donald-tru...   
 3  gossipcop-2547891536  www.dailymail.co.uk/femail/article-3499192/Do-...   
 4  gossipcop-5476631226  variety.com/2018/film/news/list-2018-oscar-nom...   
 
                                                title  \
 0  Did Miley Cyrus and Liam Hemsworth secretly ge...   
 1  Paris Jackson & Cara Delevingne Enjoy Night Ou...   
 2  Celebrities Join Tax March in Protest of Donal...   
 3  Cindy Crawford's daughter Kaia Gerber wears a ...   
 4      Full List of 2018 Oscar Nominations – Variety   
 
                                            tweet_ids  
 0  284329075902926848\t284332744559968256\t284335...  
 1  992895508267130880\t992897935418503169\t992899...  
 2  85335935

In [30]:
# Step 1: Label the datasets
fake_news_df['target'] = 0  # Fake
real_news_df['target'] = 1  # Real

# Step 2: Combine the datasets
combined_news_df = pd.concat([fake_news_df, real_news_df], ignore_index=True)

# Display the combined dataframe structure and a sample
combined_news_df.info(), combined_news_df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22140 entries, 0 to 22139
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         22140 non-null  object
 1   news_url   21871 non-null  object
 2   title      22140 non-null  object
 3   tweet_ids  20894 non-null  object
 4   target     22140 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 865.0+ KB


(None,
                      id                                           news_url  \
 0  gossipcop-2493749932  www.dailymail.co.uk/tvshowbiz/article-5874213/...   
 1  gossipcop-4580247171  hollywoodlife.com/2018/05/05/paris-jackson-car...   
 2   gossipcop-941805037  variety.com/2017/biz/news/tax-march-donald-tru...   
 3  gossipcop-2547891536  www.dailymail.co.uk/femail/article-3499192/Do-...   
 4  gossipcop-5476631226  variety.com/2018/film/news/list-2018-oscar-nom...   
 
                                                title  \
 0  Did Miley Cyrus and Liam Hemsworth secretly ge...   
 1  Paris Jackson & Cara Delevingne Enjoy Night Ou...   
 2  Celebrities Join Tax March in Protest of Donal...   
 3  Cindy Crawford's daughter Kaia Gerber wears a ...   
 4      Full List of 2018 Oscar Nominations – Variety   
 
                                            tweet_ids  target  
 0  284329075902926848\t284332744559968256\t284335...       0  
 1  992895508267130880\t992897935418503169\t9

In [31]:
# Display the counts of each target value in the combined dataset
target_counts = combined_news_df['target'].value_counts()
print(target_counts)


target
1    16817
0     5323
Name: count, dtype: int64


In [10]:
!pip install imblearn


^C


In [11]:
# Install necessary libraries
!pip install torch torchvision torchaudio
!pip install torch-geometric


   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   -------- ------------------------------- 0.3/1.3 MB ? eta -:--:--
   ---------------- ----------------------- 0.5/1.3 MB 1.2 MB/s eta 0:00:01
   ------------------------ --------------- 0.8/1.3 MB 1.2 MB/s eta 0:00:01
   -------------------------------- ------- 1.0/1.3 MB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.4 MB 1.7 MB/s eta 0:00:02
   ------------- -------------------------- 0.8/2.4 MB 1.7 MB/s eta 0:00:01
   ----------------- ---------------------- 1.0/2.4 MB 1.6 MB/s eta 0:00:01
   --------------------- ------------------ 1.3/2.4 MB 1.5 MB/s eta 0:00:01
   -------------------------- ------------- 1.6

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
# Function to clean the text data
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text, flags=re.I)  # Remove extra spaces
    text = re.sub(r'^b\s+', '', text)  # Remove prefix b if exists
    text = text.lower()  # Convert to lower case
    return text

# Apply cleaning function to the title column
combined_news_df['cleaned_title'] = combined_news_df['title'].apply(clean_text)



In [33]:
from collections import Counter
vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
tfidf_features = vectorizer.fit_transform(combined_news_df['cleaned_title'])
features = tfidf_features.toarray()
labels = combined_news_df['target'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Prepare PyTorch Geometric data
def prepare_data(X, y):
    edge_index = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long)  # Example edge index, you need to define your own based on your data
    x_tensor = torch.tensor(X, dtype=torch.float)
    y_tensor = torch.tensor(y, dtype=torch.long)
    data = Data(x=x_tensor, edge_index=edge_index, y=y_tensor)
    return data

train_data = prepare_data(X_train_smote, y_train_smote)
test_data = prepare_data(X_test, y_test)

train_loader = DataLoader([train_data], batch_size=32, shuffle=True)
test_loader = DataLoader([test_data], batch_size=32, shuffle=False)
print("Class distribution after SMOTE:", Counter(y_train_smote))

Class distribution after SMOTE: Counter({1: 13456, 0: 13456})


In [ ]:
import torch
from torch_geometric.data import Data, DataLoader
from sklearn.model_selection import train_test_split
from torch_geometric.nn import GCNConv, global_mean_pool
import torch.nn.functional as F


In [34]:

# Prepare the data splits
labels = torch.tensor(combined_news_df['target'].values, dtype=torch.long)
train_features, test_features, train_labels, test_labels = train_test_split(
    tfidf_features.toarray(), labels, test_size=0.2, random_state=42
)
train_features, val_features, train_labels, val_labels = train_test_split(
    train_features, train_labels, test_size=0.25, random_state=42  # 0.25 x 0.8 = 0.2
)


In [35]:
train_features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
!pip install tqdm

In [36]:
import torch.nn as nn

class GNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Define model
model = GNN(input_dim=1000, hidden_dim=64, output_dim=2)  # input_dim matches TF-IDF features



In [37]:
from torch_geometric.data import DataLoader
from sklearn.metrics.pairwise import cosine_similarity
# Convert numpy arrays to tensors
train_features_tensor = torch.tensor(train_features, dtype=torch.float)
val_features_tensor = torch.tensor(val_features, dtype=torch.float)
test_features_tensor = torch.tensor(test_features, dtype=torch.float)

def create_edge_index(tfidf_features, threshold=0.3):
    cosine_sim = cosine_similarity(tfidf_features)
    edges = []
    num_nodes = cosine_sim.shape[0]
    for idx1 in range(num_nodes):
        for idx2 in range(idx1 + 1, num_nodes):
            if cosine_sim[idx1, idx2] > threshold:
                edges.append((idx1, idx2))
                edges.append((idx2, idx1))  # Ensure undirected graph
    return torch.tensor(edges, dtype=torch.long).t()

edge_index_train = create_edge_index(X_train_smote)
edge_index_test = create_edge_index(X_test)

# Prepare PyTorch Geometric data
train_data = Data(x=torch.tensor(X_train_smote, dtype=torch.float), edge_index=edge_index_train, y=torch.tensor(y_train_smote, dtype=torch.long))
test_data = Data(x=torch.tensor(X_test, dtype=torch.float), edge_index=edge_index_test, y=torch.tensor(y_test, dtype=torch.long))

train_loader = DataLoader([train_data], batch_size=32, shuffle=True)
test_loader = DataLoader([test_data], batch_size=32, shuffle=False)



C:\Users\shaik\.conda\envs\tf_gpu\lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

def plot_graph(edge_index, max_edges=100):
    print("Plotting...")
    G = nx.Graph()
    edge_index = edge_index.t().cpu().numpy()
    edges = [(int(x), int(y)) for x, y in edge_index[:max_edges]]  # Limit the number of edges plotted
    G.add_edges_from(edges)
    pos = nx.spring_layout(G, seed=42)  # positions for all nodes

    nx.draw_networkx_nodes(G, pos, node_size=70)
    nx.draw_networkx_edges(G, pos, edgelist=edges, width=0.5)
    plt.show()



In [28]:
import torch
import torch.nn as nn
from tqdm import tqdm
from tqdm.auto import tqdm  # Use auto to better handle different environments like Jupyter Notebooks.

# Define the model and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Define the criterion
criterion = nn.CrossEntropyLoss()

# Training loop with managed progress bar
def train():
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc="Training Epoch Progress", leave=True)  # leave=True ensures the bar stays after completion
    for data in progress_bar:
        optimizer.zero_grad()
        output = model(data.x, data.edge_index)
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())  # Update progress bar with current loss
    return total_loss / len(train_loader)

# Evaluation function
def evaluate(loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            out = model(data.x, data.edge_index)
            pred = out.max(1)[1]
            correct += pred.eq(data.y).sum().item()
            total += data.y.size(0)
    return correct / total

# Run training and evaluation with a single progress bar for all epochs
with tqdm(range(num_epochs), desc="Overall Training Progress", leave=True) as epochs:
    for epoch in epochs:
        train_loss = train()
        test_accuracy = evaluate(test_loader)
        epochs.set_postfix(train_loss=train_loss, test_accuracy=test_accuracy)


Overall Training Progress:  68%|██████████▉     | 34/50 [00:54<00:25,  1.61s/it, test_accuracy=0.787, train_loss=0.487]


KeyboardInterrupt: 

In [41]:
# To load the model safely
model = GNN(input_dim=1000, hidden_dim=64, output_dim=2)
model.load_state_dict(torch.load('model_GNN.pth', weights_only=True))
model.eval()


GNN(
  (conv1): GCNConv(1000, 64)
  (conv2): GCNConv(64, 2)
)

In [63]:
# Function to predict and display a sample news item

def predict_sample(test_loader):
    for data in test_loader:  # Normally you would loop over the whole dataset, but here we'll just take the first batch
        out = model(data.x, data.edge_index)
        pred = out.max(1)[1]  # Get the index of the max log-probability
        break  # We only take one batch for demonstration
    
    sample_idx = 90 # Index of the sample to display
    sample_news = data.x[sample_idx]  # The features of the sample news
    actual_label = data.y[sample_idx].item()  # Actual label
    predicted_label = pred[sample_idx].item()  # Predicted label

    # Display the results
    print("Sample News Features:", sample_news)
    print("Actual Label (0-Fake, 1-Real):", actual_label)
    print("Predicted Label (0-Fake, 1-Real):", predicted_label)

# Call the function
predict_sample(test_loader)

Sample News Features: tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.